# [Total] Exposure Analysis with Kernel Density Estimation (KDE)

////////////////////////////////////////////////////////////////////////////////////
##### Author: Jay (Jiue-An) Yang
##### Organization: Health Data at Scale Collaboratory, City of Hope
##### Version: 3.0
##### Last Updated: June 04, 2021
////////////////////////////////////////////////////////////////////////////////////
***

### Requirements:
##### 1. ArcGIS Pro 2.7+
##### 2. A file directory with .csv files containing GPS points
##### 3. A file directory with raster files of env variables that needs to be processed

***
## Model Workflow as in ArcGIS Model Builder

![Alt text](KDE_Analysis_csv_08072020.svg)

--- 
# < Main script starts here >

### Step 1: Import required modules

In [ ]:
# Import required modules
import arcpy
from arcpy import env
from arcpy.sa import *
arcpy.CheckOutExtension("Spatial")

import glob
import numpy
import pandas as pd
from IPython.display import clear_output

### Step 2: Set parameters for model execution

In [ ]:
# Set environment options
arcpy.env.overwriteOutput = True

# Set methods and Point type
running_method = 'KDE'              ## Set the methods that you are running (KDE/DR/SJ)
point_type = 'Walking'              ## Set the methods that you are running (AllPoint/Stationary/Vehicle/Walking)

# Set environment variables
env.workspace = r"//full-path-to-your-workspace.gdb//"        ## full path to your ArcGIS workspace geodatabase (.gdb)
project_dir =   r"//full-path-to-your-project-folder//"       ## full path to your project folder (a directory)
gps_data_dir =  r"//full-path-to-your-GPS-dataset-folder//"   ## full path to your gps data set (a directory that contains .csv for each participant)

x_cord_name = 'lng'                  ## column name for x coordinates in point csv 
y_cord_name = 'lat'                  ## column name for y coordinates in point csv 
final_output_table = env.workspace + "/final_output_table_KDE_Walking"   ## define a name of the desired output table (in the workspace geodatabase)
final_output_table_output_dir = r"//full-path-to-output-folder//"  ## full path to your project output folder (a directory), can be same as the project_dir

# set KDE parameters
kde_cell_size = 10
kde_search_radius = 200

# Set path to some specific layers
research_area = r"//full-path-to-your-research-area-polygon//"   ## full path to your research area polygon (can be a .shp or a feature class in a geodatabase)
exposure_layer = r"//full-path-to-your-exposure-raster-layer//"  ## full path to your exposure raster layer (can be a standalone or a raster in a geodatabase)
exposure_layer_name = "Name-of-your-exposure"   ## define a name for this exposure layer, this is for the output table 

# Specify spatial reference for the analysis
spatial_ref = arcpy.SpatialReference('North America Albers Equal Area Conic')   ## define the projection that will be used for analysis

# Just a lazy way of getting the ID section from your .csv filenames 
# ex. filename is "SD000123.csv", so the range if ID is [-12:-4] from the filename --> 'SD000123'
pt_ID_start = -12
pt_ID_end  = -4

### Step 3: Define additional functions

In [ ]:
### Function to normalize the KDE raster to a defined scale

def normalize_raster(in_raster):

    # Load data, convert to numpy array
    array = arcpy.RasterToNumPyArray(in_raster)
    
    # Defined the scale for normalization
    # scale = 1    for the scale of 0-1
    # scale = 100  for the scale of 0-100
    scale = 1  
    
    # Normalize cells to 0-1
    new_array = (array - array.min()) / (array.max() - array.min()) * scale

    # Convert back to a raster
    new_raster = arcpy.NumPyArrayToRaster(
        in_array=new_array,
        lower_left_corner=in_raster.extent.lowerLeft,
        x_cell_size=in_raster.meanCellWidth,
        y_cell_size=in_raster.meanCellHeight,
    )
    
    return new_raster

### Step 4: Model execution

In [ ]:
### Clear final output table if there is data inside
if int(arcpy.GetCount_management("final_output_table")[0]) > 0:
    arcpy.DeleteRows_management("final_output_table")


### Create a log file
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
log_file_name = "log_" + dt_string + ".txt"
f = open(final_output_table_output_dir + log_file_name, "a")


### Placeholders 
processed_pts = 0
non_processed_pts = []  ## list to store non-processed PTs
    
    
### Loop through .csv files in the data directory
i = 1
total_i = len(glob.glob(gps_data_dir + "*.csv"))

for file in glob.glob(gps_data_dir + "*.csv"):
    
    clear_output(wait=True)
    # check if file is .csv
    if file[-4:] == '.csv':
        
        pt_ID = file[pt_ID_start:pt_ID_end]
        msg = "Working on : {pt} ({index}/{total})".format(pt = pt_ID, index = i, total= total_i)
        print (msg)
        
        
        ### make sure there are points in the .csv
        df = pd.read_csv(file)
        if len(df) > 10:        
            ### --------------------------------------------------------
            ### Step 1: Create feature class from CSV points
            ### --------------------------------------------------------

            arcpy.management.XYTableToPoint(file, "point_fc", x_cord_name, y_cord_name, "", arcpy.SpatialReference(4326))
            print ("Step 1: csv converted to feature class")

            ### --------------------------------------------------------
            ### Step 2: Re-project feature class
            ### --------------------------------------------------------

            arcpy.Project_management("point_fc", "point_fc_proj", spatial_ref)
            print ("Step 2: feature class re-projected to - ", spatial_ref.name)

            ### --------------------------------------------------------
            ### Step 3: Clip feature class by analysis extent
            ### --------------------------------------------------------

            arcpy.Clip_analysis("point_fc_proj", research_area, "point_fc_cliped")
            print ("Step 3: feature class clipped by research area")

            ### --------------------------------------------------------
            ### Step 4: Create KDE 
            ### --------------------------------------------------------

            KDE_raster = KernelDensity(in_features="point_fc_cliped",population_field='NONE', cell_size=kde_cell_size, search_radius=kde_search_radius, area_unit_scale_factor="SQUARE_MAP_UNITS", method="PLANAR")
            KDE_raster.save("KDE_output")
            print ("Step 4: KDE raster created")

            ### --------------------------------------------------------
            ### Step 5: Normalize the Raster to 0-1
            ### --------------------------------------------------------

            KDE_raster_normalized = normalize_raster(KDE_raster)
            arcpy.DefineProjection_management(KDE_raster_normalized, spatial_ref)
            KDE_raster_normalized.save(env.workspace + "/KDE_output_NM")
            print ("Step 5: KDE raster normalized to 0-1 scale")

            ### --------------------------------------------------------        
            ### Step 6: Set small values to Null
            ### --------------------------------------------------------

            KDE_raster_normalized_Nullfor0 = SetNull(env.workspace + "/KDE_output_NM", env.workspace + "/KDE_output_NM", "VALUE < 0.00001")
            KDE_raster_normalized_Nullfor0.save(env.workspace + "/KDE_output_NM_Null")
            print ("Step 6: Tiny values in the KDE raster set to Null")

            ### --------------------------------------------------------
            ### Step 7: Calculate Exposrue of the KDE
            ### --------------------------------------------------------

            weighted_exposure = Raster(KDE_raster_normalized_Nullfor0) * Raster(exposure_layer)
            weighted_exposure.save(env.workspace + "/Weighted_Exposure") 
            print ("Step 7: Weight exposure computed")

            ### --------------------------------------------------------
            ### Step 8: Calculate statistics for the participant
            ### --------------------------------------------------------

            exposure_statistics_table = ZonalStatisticsAsTable(research_area, "OBJECTID", weighted_exposure, env.workspace + "/exposure_table", "DATA", "ALL")
            print ("Step 8: Exposure statistics computed")
            
            ### --------------------------------------------------------
            ### Step 9: Add an new identifiet field to the table
            ### Step 10: Add PT_ID to this field
            ### --------------------------------------------------------

            arcpy.AddField_management("exposure_table", "PT_ID", "TEXT", "","", 100, "", "NULLABLE", "","")
            print ("Step 9: 'PT_ID field' added to output table")

            arcpy.CalculateField_management("exposure_table", "PT_ID", "pt_ID", "PYTHON3")    
            print ("Step 10: ID of [", pt_ID, "] added to output table")

            ### --------------------------------------------------------
            ### Step 11: Append the participant results table to the final output table
            ### --------------------------------------------------------

            arcpy.Append_management("exposure_table", "final_output_table")
            print ("Step 11: Exposure results for [", pt_ID, "] added to output table")
            
            processed_pts += 1
    
        else:  ### Write to log file to record this pt without points
            non_processed_pts.append(pt_ID)
            f.write("Participant ID: {} was not processed.\n".format(pt_ID))                    

        i+=1           

        
### --------------------------------------------------------
### Step 12: Convert Table to Pandas Dataframe
### --------------------------------------------------------

if processed_pts > 0:

    field_names = [f.name for f in arcpy.ListFields("final_output_table")]
    np_arr = arcpy.da.TableToNumPyArray("final_output_table", field_names)

    df = pd.DataFrame(data = np_arr)
    df.to_csv(final_output_table_output_dir + "Exposure_Table_" + running_method + '_' + point_type + '_' + exposure_layer_name + ".csv", index = False)

    print ("-"*30)
    print ("KDE Exposure Analysis Completed !")
    print ("Output table available at : ", final_output_table_output_dir + "Exposure_Table_" + exposure_layer_name + ".csv")
    #Final output table is saved as a .csv file in the defined [final_output_table_output_dir] location.
    
    ### Close the log file
    f.close()
    print ("Please also check the log file for non-processed participants at : {}".format(log_file_name))
    

else:     
    ### Close the log file
    f.write("-"*30)
    f.write("\nThere are no exposure resutls for the entire run.")
    f.close()
    print ("-"*30)
    print ("There are no exposure resutls for the entire run.")
    print ("Please also check the log file at : {}".format(log_file_name))       